In [1]:
import mdptoolbox as mdpt, numpy as np
import mdptoolbox.example
import MDP

Suppose, given a transition function and discount rate, we generate a random reward function over all transitions. We then sparsify the reward function by setting some proportion (e.g. 10%) of the transition values to 0. We then generate the optimal policy for said reward function (using, for instance, policy iteration). We now attempt to build a model that can predict the sparsity used to generate the optimal policy given the transition function, discount rate, and policy itself, but *not* the reward function, as otherwise the problem would be trivial.

In [43]:
### Generate a bunch of MDPs with different parameters, sparsity

NUM_MDPs = 100
NUM_STATES = 10
NUM_ACTIONS = 4

def get_transition_matrix(num_states, num_actions, generator = np.random.dirichlet):
    """
    Returns a transition matrix for a given number of states and actions
    
    Returns:
        P: (num_actions, num_states, num_states) array, where P[a, s, s'] is the probability of 
        transitioning from state s to state s' given action a
    """
    P = np.zeros((num_actions, num_states, num_states)) # (A, S, S) shape
    for a in range(num_actions):
        for s in range(num_states):
            P[a, s, :] = generator(np.ones(num_states))
    return P

def get_reward_matrix(num_states, num_actions, sparsity = 0.0, generator = np.random.normal):
    """
    Returns a reward matrix for a given number of states and actions
    [Fix 2/27/24: sparsity should be deterministic, while sparse rewards should be in random order]
    """
    num_sparse_rewards = int(sparsity * num_actions * num_states ** 2)
    rewards = np.array([(0 if i < num_sparse_rewards else generator()) for i in range(num_actions * num_states ** 2)])
    np.random.shuffle(rewards)
    return rewards.reshape((num_actions, num_states, num_states))

DISCOUNT = 0.9
EPSILON = 0.01 # roughly indicates the "skill level" of the agent
MAX_ITER = 1000

The sparsity levels generated by generate_tests are divided using arange from 0 to 1 and then scrambled randomly, meaning that in effect each sparsity level in the training and test sets is sampled uniformly from [0, 1].

In [47]:
def generate_tests(num_mdps = NUM_MDPs, sparsity_levels = None, mdp_generator = mdpt.mdp.PolicyIteration, P_generator = None):
    """
    Generate a bunch of MDPs with different sparsity levels, and return the sparsity levels and the MDPs

    Args:
        sparsity_levels: a list of sparsity levels to generate MDPs with
    Returns:
        sparsity_levels: the sparsity levels used to generate the MDPs, in the same order as the MDPs
        MDPS: an array of MDPs
    """
    sparsity_levels = sparsity_levels if sparsity_levels is not None else np.arange(num_mdps) / num_mdps
    sparsity_copy = sparsity_levels.copy() # defensive copy
    np.random.shuffle(sparsity_copy)
    MDPS = np.array([mdp_generator(
        get_transition_matrix(NUM_STATES, NUM_ACTIONS) if P_generator is None else P_generator(NUM_STATES, NUM_ACTIONS), 
        get_reward_matrix(NUM_STATES, NUM_ACTIONS, sparsity_copy[i]), 
        DISCOUNT, max_iter = MAX_ITER) 
        for i in range(num_mdps)
    ])
    return sparsity_copy, MDPS

sparsity_levels, MDPS = generate_tests()
for mdp in MDPS:
    mdp.run()
    # print(mdp.policy) # debug
# print(MDPS[0].policy) # debug

In [54]:
### Idea 1: neural network
# Thanks again ChatGPT for outlining the code structure

def fixed_P_generator(num_states, num_actions):
    """
    Returns a fixed transition matrix for a given number of states and actions
    (Ideally something we hope will give interesting results, like having some states be absorbing)
    """
    P = np.zeros((num_actions, num_states, num_states)) # (A, S, S) shape
    for a in range(num_actions):
        for s in range(num_states):
            P[a, s, :] = np.zeros(num_states)
            P[a, s, (s + 1) % num_states] = 1
    return P

def sparse_P_generator(num_states, num_actions):
    """
    Returns a sparse transition matrix for a given number of states and actions
    
    Returns:
        P: (num_actions, num_states, num_states) array, where P[a, s, s'] is the probability of 
        transitioning from state s to state s' given action a
    """
    P = np.zeros((num_actions, num_states, num_states)) # (A, S, S) shape
    for a in range(num_actions):
        for s in range(num_states):
            P[a, s, :] = np.zeros(num_states)
            P[a, s, np.random.randint(num_states)] = 1
    return P

sparsity, MDPs = generate_tests(10000, P_generator = sparse_P_generator)
# print(np.array(MDPs[0].P).shape)
training_data = [(np.array(mdp.P), mdp.discount, mdp.policy, sparsity[i]) for i, mdp in enumerate(MDPs)]

from sklearn.preprocessing import OneHotEncoder
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")

# Step 1: Feature extraction function
def extract_features(transition_function, discount_rate, optimal_policy):
    """
    Extract features from the MDP's transition function, discount rate, and optimal policy
    """
    # opt_policy = optimal_policy.reshape(-1, 1)  # Reshape for sklearn which expects 2D input

    # # Initialize the OneHotEncoder
    # encoder = OneHotEncoder(sparse=False)  # Use sparse=False to get a dense array

    # # Fit and transform
    # opt_policy_one_hot = encoder.fit_transform(opt_policy)

    features = np.concatenate((transition_function.flatten(), [discount_rate], optimal_policy.flatten()))
    # print(features.shape)
    # length A*S*S + 1 + A*S

    # Placeholder features
    # features = np.random.rand(411)

    # Policy-only features
    # features = optimal_policy
    return features

# Step 2: Data preparation (assuming you have your data in an appropriate format)
# This is a placeholder function - you would replace it with actual data loading and processing
def prepare_data(training_data):
    features = []
    labels = []
    for transition_function, discount_rate, optimal_policy, sparsity_level in training_data:
        features.append(extract_features(transition_function, discount_rate, optimal_policy))
        labels.append(sparsity_level)
    return np.array(features), np.array(labels)

# Step 3: Model selection

def build_model(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='linear')  # Linear activation for regression output
    ])

    # Num parameters: 411*64 + 64 + 64*64 + 64 + 64*64 + 64 + 64*1 + 1 = ~26500
    # Num data points: 100000
    
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mean_squared_error',  # Suitable for regression
                  metrics=['mae'])  # Mean Absolute Error as an additional metric
    # ``loss" refers to training data, ``val_loss" refers to validation data
    return model

features, labels = prepare_data(training_data)
# Example: features shape is (num_samples, num_features), adjust 'input_dim' accordingly
input_dim = features.shape[1]  # Assuming 'features' is already defined and preprocessed

model = build_model(input_dim)

# Training the model
model.fit(features, labels, epochs=100, validation_split=0.2, verbose = 1, 
          callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])

# Don't forget to preprocess your new data before making predictions
# predicted_sparsity = model.predict(new_features)

# Step 5: Prediction function
def predict_sparsity(transition_function, discount_rate, optimal_policy):
    features = extract_features(transition_function, discount_rate, optimal_policy).reshape(1, -1)
    predicted_sparsity = model(features) # more efficient than .predict() for single samples
    return predicted_sparsity

# Testing model
test_sparsity, test_MDPs = generate_tests(10000, P_generator=sparse_P_generator)
test_data = [(np.array(mdp.P), mdp.discount, mdp.policy) for mdp in (test_MDPs)]
NUM_TESTS = 1000
mse = np.zeros(min(NUM_TESTS, len(test_data)))

for i in range(min(NUM_TESTS, len(test_data))):
    transition_function, discount_rate, optimal_policy = test_data[i]
    prediction = predict_sparsity(transition_function, discount_rate, optimal_policy)[0][0]
    mse[i] = (prediction - test_sparsity[i])**2
    # print(f"Predicted sparsity level for MDP {i+1}: {prediction}, actual sparsity level: {test_sparsity[i]}, Squared error: {mse[i]}")

print(f"Mean squared error: {np.mean(mse)}, sample size: {min(NUM_TESTS, len(test_data))}")
print("Expected squared error: when x, y ~ U[0, 1], E[(x-y)^2] = 1/12 = 0.0833...")

Epoch 1/100
250/250 [==============================] - 5s 9ms/step - loss: 0.0802 - mae: 0.2263 - val_loss: 0.0429 - val_mae: 0.1716
Epoch 2/100
250/250 [==============================] - 1s 6ms/step - loss: 0.0462 - mae: 0.1752 - val_loss: 0.0422 - val_mae: 0.1726
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 0.0404 - mae: 0.1647 - val_loss: 0.0377 - val_mae: 0.1607
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.0362 - mae: 0.1554 - val_loss: 0.0431 - val_mae: 0.1753
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.0331 - mae: 0.1480 - val_loss: 0.0398 - val_mae: 0.1670
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.0304 - mae: 0.1418 - val_loss: 0.0406 - val_mae: 0.1676
Predicted sparsity level for MDP 0: 0.6853408813476562, actual sparsity level: 0.7872, Squared error: 0.01037527434527874
Predicted sparsity level for MDP 1: 0.8440966606140137, actual sparsity level: 0

With ten actions:
- As a control, when the input layer (with same dimension as transition_function + discount rate + optimal policy) is randomized, MSE = ~0.115
- I should also note that I'm choosing hyperparameters here in a rather unprincipled way by guess-timating their effects on the model
- The loss seems to settle around 0.033 after ~20% into each epoch when given 10^5 training points 
    - Maybe there's some sort of irreducible randomness going on when you randomize the reward function and don't pass it into the models?
- When reward is defined over (A, S, S') (i.e. all transitions) instead of state-action pairs, loss rises to ~0.076, i.e. basically random

With 100 actions:
- ~~Model does slightly better (now ~0.028); maybe patterns in MDP/sparsity become more apparent with more states?~~ There was an error in how I calculated the MSE here, so now I'm not sure
- In terms of computation, generating the MDPs takes a lot longer than training the model
    - Increasing epsilon doesn't improve MDP generation/solving time (it actually makes it worse for some reason); I assume then that most of the calculation is in generating the MDPs themselves

With deterministic MDPs: 
- I initially had an error that made the validation and test loss very different; it turns out that my test set was from denser MDPs, which actually tells us that the those respective models are fundamentally different
- Even when the reward function is defined over transitions, the models with deterministic MDPs approach ~0.033 (as we would expect).

In [77]:
### Idea 2: Multiple linear regression 

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

def extract_features(transition_function, discount_rate, optimal_policy):
    """
    Extract features from the MDP's transition function, discount rate, and optimal policy
    """
    # features = np.concatenate((transition_function.flatten(), [discount_rate], optimal_policy.flatten()))
    features = optimal_policy
    return features

sparsity, MDPs = generate_tests(10000, P_generator = sparse_P_generator)
# print(np.array(MDPs[0].P).shape)
training_data = [(np.array(mdp.P), mdp.discount, mdp.policy, sparsity[i]) for i, mdp in enumerate(MDPs)]
# print(sparsity)
features, labels = prepare_data(training_data)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Create a model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean squared error: {mse}")
print("Expected squared error: when x, y ~ U[0, 1], E[(x-y)^2] = 1/12 = 0.0833...")
print(f"Mean absolute error: {mae}")

model.coef_

Mean squared error: 0.03527557718220251
Expected squared error: when x, y ~ U[0, 1], E[(x-y)^2] = 1/12 = 0.0833...
Mean absolute error: 0.14856711803719877


array([-0.03914735, -0.03764345, -0.03981131, -0.04036957, -0.04082197,
       -0.04326215, -0.03952871, -0.03927806, -0.03984426, -0.0402734 ])

- Interesting! When I increased the number of states from 10 to 100 and *decreased* the number of training data points from 10^5 to 10^4, test loss *decreased* from ~0.033 to ~0.014 and stayed that way with training data = 10^3
- When I increased the number of actions from 4 to 40, both training methods got higher MSE (~0.07) with 1000 data samples

In [99]:
### Idea 3: hand-crafted features

from sklearn.preprocessing import OneHotEncoder, normalize
def extract_features(transition_function, discount_rate, optimal_policy):
    """
    Extract features from the MDP's transition function, discount rate, and optimal policy
    Test features that I think might be relevant
    - Sparsity of the transition function
    - Number and length of loops
    - Distance to loops/absorbing states
    - Number of absorbing states
    - Number of states that are never visited
    - Number of states with lots of outward transitions
    """
    transition_sparsity = np.mean(transition_function == 0)
    num_loops = 0
    loop_lengths = []
    for s in range(transition_function.shape[1]):
        exists_loop = [transition_function[a, s, s] > 0.5 for a in range(transition_function.shape[0])]
        a = np.argmax(exists_loop)
        if exists_loop[a]:
            # checking if there exists an action that leads to the same state with probability > 0.5
            num_loops += 1
            loop_length = 1
            next_state = np.argmax(transition_function[a, s, :])
            while next_state != s:
                loop_length += 1
                next_state = np.argmax(transition_function[a, next_state, :])
            loop_lengths.append(loop_length)
    avg_loop_length = np.mean(loop_lengths) if len(loop_lengths) > 0 else 0

    # Policy features
    encoder = OneHotEncoder(sparse = False, drop = 'first')
    # Drop first to avoid multicollinearity, large coefficients
    encoder.fit(np.arange(NUM_ACTIONS).reshape(-1, 1))
    # print(encoder.categories_)
    # print(optimal_policy)
    optimal_policy = encoder.transform(optimal_policy.reshape(-1, 1)).reshape(-1)
    features = optimal_policy
    # in features[0:4] we have the one-hot encoding of the first action (only one of them is 1)

    # features = normalize(np.array([transition_sparsity, num_loops, avg_loop_length]).reshape(-1, 1), axis=0).reshape(-1)
    # features = np.append(np.array([transition_sparsity, num_loops, avg_loop_length]), optimal_policy)
    # features = np.concatenate((transition_function.flatten(), [discount_rate]))
    # print(features)
    return features

### Neural network
# Data generation
sparsity, MDPs = generate_tests(10000, P_generator = sparse_P_generator)
# print(np.array(MDPs[0].P).shape)
training_data = [(np.array(mdp.P), mdp.discount, mdp.policy, sparsity[i]) for i, mdp in enumerate(MDPs)]

features, labels = prepare_data(training_data)
# Example: features shape is (num_samples, num_features), adjust 'input_dim' accordingly
input_dim = features.shape[1]  # Assuming 'features' is already defined and preprocessed

model = build_model(input_dim)

# Training the model
model.fit(features, labels, epochs=100, validation_split=0.2, verbose = 1, 
          callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])

features, labels = prepare_data(training_data)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

### Multiple linear regression
# Create a model
model_lin = LinearRegression()
model_lin.fit(X_train, y_train)

# Make predictions
y_pred = model_lin.predict(X_test)

# Evaluate the model
print([(y_test[i], y_pred[i], sum(X_test[i] * model_lin.coef_) + model_lin.intercept_) for i in range(10)])
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("Linear regression:")
print(f"Mean squared error: {mse}")
print("Expected squared error: when x, y ~ U[0, 1], E[(x-y)^2] = 1/12 = 0.0833...")
print(f"Mean absolute error: {mae}")

# show the coefficients
model_lin.coef_, model_lin.intercept_

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.0849 - mae: 0.2356 - val_loss: 0.0373 - val_mae: 0.1580
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.0455 - mae: 0.1742 - val_loss: 0.0331 - val_mae: 0.1456
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.0391 - mae: 0.1601 - val_loss: 0.0317 - val_mae: 0.1414
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.0371 - mae: 0.1551 - val_loss: 0.0320 - val_mae: 0.1428
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.0362 - mae: 0.1537 - val_loss: 0.0310 - val_mae: 0.1383
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.0347 - mae: 0.1500 - val_loss: 0.0320 - val_mae: 0.1399
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.0345 - mae: 0.1495 - val_loss: 0.0308 - val_mae: 0.1398
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - 

(array([-0.07525723, -0.09697196, -0.10597604, -0.06883386, -0.09370967,
        -0.09615482, -0.07406292, -0.09224135, -0.09017759, -0.07027454,
        -0.10127237, -0.08843233, -0.07194101, -0.09175604, -0.10295307,
        -0.07514327, -0.09457876, -0.09889793, -0.08458914, -0.10037591,
        -0.0947758 , -0.07092147, -0.08808789, -0.10025986, -0.06949765,
        -0.09143081, -0.10136684, -0.08136231, -0.10208819, -0.10089903]),
 0.9766419600896789)

In [98]:
# Find difference between coefficients (since they are so large)
[np.max(model_lin.coef_[4 * i : 4 * (i+1)]) - np.min(model_lin.coef_ [4 * i : 4 * (i+1)]) for i in range(10)]

[0.11004638671875,
 0.093109130859375,
 0.10845947265625,
 0.104034423828125,
 0.092620849609375,
 0.10546875,
 0.0908203125,
 0.09075927734375,
 0.1014404296875,
 0.0977630615234375]

- Tossing in only features of the MDP (loop length, etc.) doesn't seem to help (~near-random MSE), but including the policy immediately jumps to 0.033 again
- On linear regression when one-hot encoding is applied to just the optimal policy, the coefficients are the same for every chunk of four elements, and they're all very large (magnitude ~1E10-5E12) for some reason
- Training on just the transition function + discount rate gives basically random results

In [ ]:
""" Saving coefficient outputs
First run:
array([-1.30675583e+11, -1.30675583e+11, -1.30675583e+11, -1.30675583e+11,
        2.25297886e+11,  2.25297886e+11,  2.25297886e+11,  2.25297886e+11,
       -1.25434170e+10, -1.25434170e+10, -1.25434170e+10, -1.25434170e+10,
        7.59830556e+11,  7.59830556e+11,  7.59830556e+11,  7.59830556e+11,
        2.88124182e+11,  2.88124182e+11,  2.88124182e+11,  2.88124182e+11,
       -2.20636912e+12, -2.20636912e+12, -2.20636912e+12, -2.20636912e+12,
        1.17074647e+12,  1.17074647e+12,  1.17074647e+12,  1.17074647e+12,
       -1.08094503e+12, -1.08094503e+12, -1.08094503e+12, -1.08094503e+12,
        4.57643503e+11,  4.57643503e+11,  4.57643503e+11,  4.57643503e+11,
       -1.72875563e+12, -1.72875563e+12, -1.72875563e+12, -1.72875563e+12])
Second run:
array([ 6.42943536e+10,  6.42943536e+10,  6.42943536e+10,  6.42943536e+10,
        4.38543769e+10,  4.38543769e+10,  4.38543769e+10,  4.38543769e+10,
        1.25801626e+11,  1.25801626e+11,  1.25801626e+11,  1.25801626e+11,
        2.11727524e+11,  2.11727524e+11,  2.11727524e+11,  2.11727524e+11,
        2.01145141e+09,  2.01145141e+09,  2.01145141e+09,  2.01145141e+09,
       -7.43402751e+11, -7.43402751e+11, -7.43402751e+11, -7.43402751e+11,
        8.19022461e+11,  8.19022461e+11,  8.19022461e+11,  8.19022461e+11,
       -2.38391058e+12, -2.38391058e+12, -2.38391058e+12, -2.38391058e+12,
        5.12756087e+11,  5.12756087e+11,  5.12756087e+11,  5.12756087e+11,
       -1.69323492e+11, -1.69323492e+11, -1.69323492e+11, -1.69323492e+11])
"""